In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict

In [2]:
path = '/Users/pranav/nfl_betting_system/data_processing_files/base_game_info.csv'
df = pd.read_csv(path)

In [3]:
df.rename(columns={'schedule_season':'season', 'schedule_date':'date', 'schedule_week':'week'}, inplace=True)

In [4]:
#initialize all the new stats we want to add to the dataframe to empty lists except for win% which is set to 0s
df['home_team_GP'] = ''
df['home_team_wins']= ''
df['home_win%'] = np.zeros(len(df))
df['home_team_ppg'] = ''
df['home_team_oppg'] = ''
df['away_team_GP'] = ''
df['away_team_wins'] = ''
df['away_win%'] = np.zeros(len(df))
df['away_team_ppg'] = ''
df['away_team_oppg'] = ''

In [5]:
#warn us about the possibility that the function is implemented on a slice/copy of the dataframe rather than 
#the original in which case would result in the return of the original (unchanged)
pd.options.mode.chained_assignment = 'warn'

def getRecord(dataframe):
    #list: [Games Played, Wins, Total Points, Total Opp Points]
    lst = lambda:list([0]*4)
    #create a nested dictionary of a list of size 4 to hold all our values
    #use lambda to allow defaultdict to work on a callable function 
    store = defaultdict(lambda: defaultdict(lst))
    
    for idx in range(len(df)):
        #this is to lock a variable 'row' for each row in the dataframe (as a reference)
        row = dataframe.iloc[idx]
        
        #follow a particular order to maximize cache hits and temporal storage
        #first, update the dataframe with the values from our nested dictionary
        #USE .LOC as the spec makes sure it edits the original dataframe and not a copy/slice
        
        #####################################################################################
        #HOME TEAM:
        
        dataframe.loc[idx, 'home_team_GP'] = store[row['season']][row['home_team_id']][0]
        dataframe.loc[idx, 'home_team_wins'] = store[row['season']][row['home_team_id']][1]
        dataframe.loc[idx, 'home_team_ppg'] = store[row['season']][row['home_team_id']][2]
        
        if dataframe.loc[idx, 'home_team_GP'] != 0:
            dataframe.loc[idx, 'home_team_ppg'] /= dataframe.loc[idx, 'home_team_GP']
            
        dataframe.loc[idx, 'home_team_oppg'] = store[row['season']][row['home_team_id']][3]
        
        if dataframe.loc[idx, 'home_team_GP'] != 0:
            dataframe.loc[idx, 'home_team_oppg'] /= dataframe.loc[idx, 'home_team_GP']
            
        #####################################################################################
        #AWAY TEAM:
            
        dataframe.loc[idx, 'away_team_GP'] = store[row['season']][row['away_team_id']][0]
        dataframe.loc[idx, 'away_team_wins'] = store[row['season']][row['away_team_id']][1]
        dataframe.loc[idx, 'away_team_ppg'] = store[row['season']][row['away_team_id']][2]
        
        if dataframe.loc[idx, 'away_team_GP'] != 0:
            dataframe.loc[idx, 'away_team_ppg'] /= dataframe.loc[idx, 'away_team_GP']
            
        dataframe.loc[idx, 'away_team_oppg'] = store[row['season']][row['away_team_id']][3]
        
        if dataframe.loc[idx, 'away_team_GP'] != 0:
            dataframe.loc[idx, 'away_team_oppg'] /= dataframe.loc[idx, 'away_team_GP']
            
            
        #second, update our nested dictionary 
        store[row['season']][row['home_team_id']][0] += 1
        store[row['season']][row['home_team_id']][2] += row['score_home']
        store[row['season']][row['home_team_id']][3] += row['score_away']
        store[row['season']][row['away_team_id']][0] += 1
        store[row['season']][row['away_team_id']][2] += row['score_away']
        store[row['season']][row['away_team_id']][3] += row['score_home']
        
        #only add to winner's column if the game wasn't a tie
        if row['winner'] != 'TIE':
            store[row['season']][row['winner']][1] += 1
            
    return dataframe

df = getRecord(df)

In [6]:
ppgs = ['home_team_ppg', 'home_team_oppg', 'away_team_ppg', 'away_team_oppg']
for ppg in ppgs:
    df[ppg] = np.round(df[ppg].astype(float), 2)

In [7]:
#similar function to getRecords, iterate through df and set win% when GP != 0
def percentages(dataframe):
    df = dataframe
    for idx in range(len(df)):
        if df.loc[idx, 'home_team_GP'] != 0:
            df.loc[idx, 'home_win%'] = np.round((df.loc[idx, 'home_team_wins']/df.loc[idx, 'home_team_GP']), 3)
        if df.loc[idx, 'away_team_GP'] != 0:
            df.loc[idx, 'away_win%'] = np.round((df.loc[idx, 'away_team_wins']/df.loc[idx, 'away_team_GP']), 3)
    return df
df = percentages(df)

In [8]:
#calculate Δpoints for each team (ppg - oppg)
df.insert(23, 'home_p_diff', np.round(df['home_team_ppg']-df['home_team_oppg'], 2))
df['away_p_diff'] = np.round(df['away_team_ppg']-df['away_team_oppg'], 2)

In [9]:
df = df.drop('date', axis=1)
df

,season,week,home_team_id,team_home,score_home,score_away,team_away,away_team_id,team_favorite_id,spread_favorite,...,home_win%,home_team_ppg,home_team_oppg,home_p_diff,away_team_GP,away_team_wins,away_win%,away_team_ppg,away_team_oppg,away_p_diff
0,2000,1,ATL,Atlanta Falcons,36,28,San Francisco 49ers,SF,ATL,-6.5,...,0.000,0.00,0.00,0.00,0,0,0.000,0.00,0.00,0.00
1,2000,1,BUF,Buffalo Bills,16,13,Tennessee Titans,TEN,BUF,-1.0,...,0.000,0.00,0.00,0.00,0,0,0.000,0.00,0.00,0.00
2,2000,1,CLE,Cleveland Browns,7,27,Jacksonville Jaguars,JAX,JAX,-10.5,...,0.000,0.00,0.00,0.00,0,0,0.000,0.00,0.00,0.00
3,2000,1,DAL,Dallas Cowboys,14,41,Philadelphia Eagles,PHI,DAL,-6.0,...,0.000,0.00,0.00,0.00,0,0,0.000,0.00,0.00,0.00
4,2000,1,GB,Green Bay Packers,16,20,New York Jets,NYJ,GB,-2.5,...,0.000,0.00,0.00,0.00,0,0,0.000,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5099,2019,17,MIN,Minnesota Vikings,19,21,Chicago Bears,CHI,CHI,-4.5,...,0.667,25.87,18.80,7.07,15,7,0.467,17.27,18.60,-1.33
5100,2019,17,NE,New England Patriots,24,27,Miami Dolphins,MIA,NE,-17.0,...,0.800,26.40,13.20,13.20,15,4,0.267,18.60,31.33,-12.73
5101,2019,17,NYG,New York Giants,17,34,Philadelphia Eagles,PHI,PHI,-3.5,...,0.267,21.60,27.80,-6.20,15,8,0.533,23.40,22.47,0.93
5102,2019,17,SEA,Seattle Seahawks,21,26,San Francisco 49ers,SF,SF,-3.5,...,0.733,25.60,24.80,0.80,15,12,0.800,30.20,19.27,10.93


In [10]:
savePath = '/Users/pranav/nfl_betting_system/data_processing_files/advanced_game_info.csv'
df.to_csv(savePath, index=False)